In [14]:
from pymongo import MongoClient, DESCENDING, ASCENDING
from bson.objectid import ObjectId
import csv
from tqdm import tnrange, tqdm_notebook
import pandas as pd

from gensim.corpora import Dictionary
from gensim import models
import numpy as np


db = MongoClient().thesis.final_db
db_raw = MongoClient().thesis.merged

In [15]:
db.count(), db_raw.count()

(33875, 33877)

In [16]:
db.find_one()["comments"][0][-1]

0

In [17]:
texts = [doc["content"] for doc in db.find()]

In [18]:
dct = Dictionary(texts)

In [19]:
len(dct)

118718

In [20]:
def find_infrequient(documents):
    """ На входе принимаем токенизированный список документов
    вида [[токен1, токен2], [токен1, токен3]]
    Находим слова, которые встречаются меньше определённого количества раз
    98262 токенов, которые встречаются 2 или 1 раз
    49279 -- 1 раз
    54552 токенов, которые встречаются только в одном документе
    всего 118806
    """

    from collections import Counter

    c = Counter(token for document in documents for token in document)

    infrequient = []

    # Single line: [[token for token in document if c[token] > 1] for document in documents]
    for document in documents:
        for token in document:
            if c[token] <= 1:
                #print "Токен [{0}] встречается {1} раз".decode("utf8").format(token, c[token])
                infrequient.append(token)

    print("Общее количество токенов {0}. Найдёно {1} редких токенов".format(len(c), len(infrequient)))
    return infrequient

In [21]:
solo_tokens = find_infrequient(texts)

Общее количество токенов 118718. Найдёно 49277 редких токенов


In [22]:
solo_ids = [_id for _id, token in dct.iteritems() if token in solo_tokens]

In [23]:
dct.filter_tokens(solo_ids)

In [24]:
dct.compactify()

In [25]:
len(dct)

69441

In [26]:
corpus = [dct.doc2bow(text) for text in texts]

In [27]:
models_list = []

for m_num in range(2):
    lda = models.LdaMulticore(corpus=corpus,
                              id2word=dct,
                              num_topics=50,
                              workers=3)
    models_list.append(lda)

In [28]:
models_list[0].print_topics()

[(45,
  '0.011*"омск" + 0.009*"конкурс" + 0.008*"россия" + 0.004*"день" + 0.003*"получить" + 0.003*"человек" + 0.003*"дорога" + 0.003*"новый" + 0.003*"область" + 0.003*"работа"'),
 (13,
  '0.014*"омск" + 0.012*"омичка" + 0.008*"градус" + 0.007*"область" + 0.006*"температура" + 0.006*"россия" + 0.005*"команда" + 0.004*"ребёнок" + 0.004*"матч" + 0.004*"днём"'),
 (43,
  '0.015*"россия" + 0.008*"украина" + 0.006*"омск" + 0.004*"район" + 0.004*"выбор" + 0.004*"область" + 0.003*"глава" + 0.003*"страна" + 0.003*"рубль" + 0.003*"место"'),
 (18,
  '0.015*"омск" + 0.012*"строительство" + 0.008*"работа" + 0.006*"объект" + 0.005*"район" + 0.005*"улица" + 0.005*"область" + 0.004*"мостовик" + 0.004*"компания" + 0.004*"дом"'),
 (33,
  '0.015*"омск" + 0.006*"россия" + 0.005*"автомобиль" + 0.005*"область" + 0.004*"рубль" + 0.004*"человек" + 0.004*"время" + 0.004*"омич" + 0.004*"город" + 0.003*"улица"'),
 (44,
  '0.025*"омск" + 0.014*"область" + 0.010*"россия" + 0.008*"губернатор" + 0.008*"регион" + 0.0

In [30]:
new_dataset = {}

for num, doc in enumerate(db.find()):
    print(num, end="\r")
    
    bow = dct.doc2bow(doc["content"])
    probs1 = models_list[0].get_document_topics(bow, minimum_probability=0)
    probs2 = models_list[1].get_document_topics(bow, minimum_probability=0)
    
    try:
        raw_doc = db_raw.find_one({"url": doc["url"]})
    except Exception as err:
        if doc["source"] == "bk55_preprocessed":
            raw_doc = MongoClient().thesis.bk55_preprocessed.find_one({"url": doc["url"]})
        if doc["source"] == "gorod55":
            raw_doc = MongoClient().thesis.gorod55.find_one({"url": doc["url"]})
        if doc["source"] == "ngs55":
            raw_doc = MongoClient().thesis.ngs55.find_one({"url": doc["url"]})
        if doc["source"] == "omskinform":
            raw_doc = MongoClient().thesis.omskinform.find_one({"url": doc["url"]})
        else:
            break
        
    try:
        d = {
            "_id": str(doc["_id"]),
            "topics1": probs1,
            "topics2": probs2,
            "title": doc["title"],
            "source": doc["url"],
            "content": raw_doc["content"],
            "comments": doc["comments"]
        }
        new_dataset[doc["url"]] = d
    except Exception as err:
        print(err)
        print(doc)

In [31]:
len(new_dataset)

33871

In [32]:
new_dataset["http://www.bk55.ru/news/article/37963/"]

{'_id': '5489d17037a808115d1e6c24',
 'comments': [['в настоящий время он давать показание. анна давыдова источник: http://bk55.ru/news/article/37963/ как до-о-о-олго он они давать…\n',
   0,
   0]],
 'content': 'Женщина лишилась золотых изделий на сумму около 7 тысяч рублей прямо в салоне автобуса. Вчера, 31 августа, в полицию поступило сообщение о грабеже. Около 17 часов на остановке по улице Б. Хмельницкого прямо в салоне автобуса ограбили кондуктора одного из омских транспортных предприятий. По словам пострадавшей, неизвестный похитил у нее золотые изделия общей стоимостью почти 7 тысяч рублей. Подозреваемый вскоре был задержан, им оказался ранее судимый безработный 30-летний омич. В настоящее время он дает показания. ',
 'source': 'http://www.bk55.ru/news/article/37963/',
 'title': 'В Омске пассажир автобуса украл у кондуктора золото',
 'topics1': [(0, 0.00029850742),
  (1, 0.00029850742),
  (2, 0.00029850742),
  (3, 0.00029850742),
  (4, 0.00029850742),
  (5, 0.00029850742),
  (6,

In [33]:
from collections import defaultdict

In [34]:
topic1_results = {}
topic2_results = {}

for url, doc in tqdm_notebook(new_dataset.items()):
    sent_scores = [comment[-1] for comment in doc["comments"]]
    comments_num = len(doc["comments"])
    
    if comments_num:
        avg_comment_sent = sum(sent_scores)/comments_num
        avg_comment_std = np.std(sent_scores)

        for topic, prob in doc["topics1"] :
            topic1_results.setdefault(topic, defaultdict(int))

            topic1_results[topic]["comments_num"] += prob * comments_num           
            topic1_results[topic]["avg_comment_sent"] += prob * avg_comment_sent
            topic1_results[topic]["avg_comment_std"] += prob * avg_comment_std

        for topic, prob in doc["topics2"]:
            topic2_results.setdefault(topic, defaultdict(int))

            topic2_results[topic]["comments_num"] += prob * comments_num           
            topic2_results[topic]["avg_comment_sent"] += prob * avg_comment_sent
            topic2_results[topic]["avg_comment_std"] += prob * avg_comment_std

HBox(children=(IntProgress(value=0, max=33871), HTML(value='')))

In [35]:
topic1_sum = {}
topic2_sum = {}

for url, doc in tqdm_notebook(new_dataset.items()):
    for topic, prob in doc["topics1"] :
        topic1_sum.setdefault(topic, 0)

        topic1_sum[topic] += prob          

    for topic, prob in doc["topics2"]:
        topic2_sum.setdefault(topic, 0)

        topic2_sum[topic] += prob 

HBox(children=(IntProgress(value=0, max=33871), HTML(value='')))

In [36]:
topcs_sum_df = pd.concat([pd.DataFrame.from_dict(topic1_sum, orient="index").transpose(),
           pd.DataFrame.from_dict(topic2_sum, orient="index").transpose()],
           ignore_index=True
         ).transpose()
topcs_sum_df.head()

,0,1
0,182.287492,231.052475
1,171.369305,544.965852
2,450.864556,247.342021
3,394.038267,969.297061
4,140.591960,253.321139


In [37]:
df1 = pd.DataFrame(topic1_results)

In [38]:
df2 = pd.DataFrame(topic2_results)
df2

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
avg_comment_sent,43.754400,91.445968,41.403743,145.095126,32.789199,64.067696,73.500323,55.562704,147.130359,363.712899,...,190.610999,43.738161,43.336822,112.455085,124.976907,121.947127,43.861095,140.081289,44.616291,27.565969
avg_comment_std,70.538766,203.360449,92.803585,284.849288,65.303276,185.529653,143.659477,155.873252,282.909634,751.205148,...,239.582621,162.719333,86.495856,233.477277,185.572545,288.795468,80.334863,279.175536,109.653471,62.553785
comments_num,1523.314242,4347.826095,2195.682198,5281.334706,1394.340277,5452.999675,3435.863821,4066.090088,4913.860759,17694.486395,...,4409.566284,3335.970975,1827.974401,5871.065389,3236.899152,5995.992490,1685.909326,5809.104458,2361.965533,1354.109604


In [63]:
models_metrics_writer = pd.ExcelWriter("models_metrics.xlsx")

topcs_sum_df.to_excel(models_metrics_writer, "prob_sum")
df1.transpose().to_excel(models_metrics_writer, "model0_metrics")
df2.transpose().to_excel(models_metrics_writer, "model1_metrics")

models_metrics_writer.save()

In [46]:
def top_docs_for_topic(data, model_num, t_num, limit=50):
    return list(
        sorted(
            data.items(),
            key=lambda doc: doc[1][f"topics{model_num+1}"][t_num],
            reverse=True
        )[:limit],
    )      

In [41]:
top_docs_for_topic(new_dataset, 1, 0)

[('http://www.bk55.ru/news/article/32811/',
  {'_id': '5489d37c37a808115d1e7d61',
   'comments': [['35% заявлять что почему-то рваться, а сколько заявлять, что еще и гнуться?))очень информативный исследование…\n',
     0,
     0]],
   'content': 'Известное исследовательское агентство проанализировало недостатки неправильного использования контрацептивов в России. Агентством был проведен опрос, согласно результатам которого россияне считают презерватив самым распространенным средством контрацепции. Выяснилось, что 91% опрошенных считают, что умеют правильно пользоваться этим изделием, а 76% планируют его как средство для предохранения в течение ближайших трех лет. Однако 35% опрошенных сообщили, что презерватив почему-то рвется. Из них половина сослалась на брак производства и лишь четверть призналась в неправильном его использовании. По данным исследований мировой уровень «неудачного изделия» составляет всего лишь 28%. Такое неудачное использование данного средства контрацепции приводи

In [45]:
model_num = 0
f"topics{model_num+1}"

'topics1'

In [61]:
for model_num in [0,1]:
    with open(f"topics_interpretation{model_num}.csv", "wt") as f:
        fieldnames = []
        csvwriter = csv.writer(f, delimiter=";")
        csvwriter.writerow(["id", "topic_terms", "text", "source", "probability", "comments"])



        for t_num in tnrange(50):
            topic_terms = models_list[model_num].print_topic(t_num, topn=20)

            csvwriter.writerow([str(t_num) + "_0_0", topic_terms, "", "", "", ""])
            for d_num, doc in enumerate(top_docs_for_topic(new_dataset, 0, t_num)):
                d_num += 1
                text = doc[1]["title"] + " " + doc[1]["content"]
                text = text.replace(";", ".").replace("\n", " ").replace("\t", " ").replace("\r", " ")
                try:
                    csvwriter.writerow(["{}_{}_0".format(t_num, d_num), "", text, doc[0], doc[1][f"topics{model_num+1}"][t_num], ""])
                except UnicodeEncodeError as err:
                    pass


                for c_num, comment in enumerate(doc[1]["comments"]):
                    c_num += 1
                    try:
                        csvwriter.writerow(["{}_{}_{}".format(t_num, d_num, c_num), "", "", "", "", "", comment[0].replace("\n", " ").replace("\t", " ").replace("\r", " ")])
                    except UnicodeEncodeError as err:
                        pass
                    
    pd.DataFrame(models_list[model_num].print_topics(num_topics=-1, num_words=20)).to_excel(f"terms_for_model_{model_num}.xlsx")

HBox(children=(IntProgress(value=0, max=50), HTML(value='')))

HBox(children=(IntProgress(value=0, max=50), HTML(value='')))

In [56]:
pd.DataFrame(models_list[model_num].print_topics(num_topics=-1, num_words=20)).to_excel(f"terms_fpr_model_{model_num}.xlsx")